In [48]:
# Vikram & Sahana
# Jan 17 2022
# Classical Machine Learning Model V3
# Detecting Severity of Alzheimer's with brain scans more accurately

# Differences from V2
#   Including more congitive impairments
#   Including 3 slices at the Quartiles

In [3]:
from tqdm import tqdm
import os
import cv2
import numpy as np
from random import shuffle
import matplotlib.pyplot as plt
import tflearn
import nibabel as nib
nib.imageglobals.logger.level = 40
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, image_preloader
from tflearn.metrics import Accuracy
import numpy as np
import pandas as pd
import skimage, os
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
import numpy as np
from glob import glob
from skimage.io import imread
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from time import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
import numpy as np
import argparse
import locale
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
def process_images():
    print("Processing Images...")
    df = pd.read_csv(CSV_DIR) # reads in csv file
    df["Downloaded"] = "" # creates new Downloaded column
    
    
    for index, row in df.iterrows(): # goes through every row
        
        # CONNECTING EACH IMAGE TO ROW IN THE 
        id = row['Image ID']
        for i in range(len(images_set)): # checks through all the images
            if "I%s" % id in image_info[i][0]:
                # if the image id is in the file name of the image being checked
                df.at[index,'Downloaded'] = i # store the index
                image_info[i][1] = True
        
        # CHANGING RESEARCH GROUP TO A NUMBER
        #if (row['Research Group'] == "AD"): df.at[index,'Research Group'] = 1
        #elif (row['Research Group'] == "CN"): df.at[index,'Research Group'] = 0
        #else: 
        #    df.at[index,'Research Group'] = 2
            
        if (row['Research Group'] in RG_CLASSIFICATIONS.keys()): 
            df.at[index,'Research Group'] = RG_CLASSIFICATIONS[row['Research Group']]
        else:
            df.at[index,'Research Group'] = -1
        # CHANGING ANY COL,ROW WHERE IT SAYS NAN
        for col in df:
            if index > 0 and pd.isnull(row[col]):
                if row["Subject ID"] == df.at[index-1,"Subject ID"]:
                    df.at[index, col] = df.at[index-1,col]
        
#         if row['Subject ID'] == df.at[index-1,"Subject ID"] and row['Research Group'] != df.at[index-1,"Research Group"]:
#             print(index, )
        
    df.replace([np.inf, -np.inf], np.nan, inplace=True)   
    df.dropna(inplace=True) 
    
    
    df2 = df.loc[df['Downloaded'] != ""]
    df3 = df2.loc[df2['Research Group'] != -1]
    print("Processed!")
    return df3

def process_lattributes(df, dat):
    print("Processing Scan Attributes...")
    cs = MinMaxScaler()
    continuous = cs.fit_transform(dat[INCLUDE_DATA_NUMBERS])
    categoricals = []
    for i in INCLUDE_DATA_CATEGORIES:
        zipBinarizer = LabelBinarizer().fit(df[i])
        categorical = zipBinarizer.transform(dat[i])
        categoricals.append(categorical)
    
    x = np.hstack([continuous, *categoricals])
    print("Processed!")
    return x

def get_data():
    print("Getting Image Data...")
    path = IMAGE_DIR
    images = []
    image_info = []
    for fol1 in tqdm(os.listdir(path), disable=True):
        if fol1 == ".DS_Store": continue
        path1 = os.path.join(path, fol1)
        for fol2 in tqdm(os.listdir(path1), disable=True):
            if fol2 == ".DS_Store": continue
            path2 = os.path.join(path1, fol2)
            for fol3 in tqdm(os.listdir(path2), disable=True):
                if fol3 == ".DS_Store": continue
                path3 = os.path.join(path2, fol3)
                for fol4 in tqdm(os.listdir(path3), disable=True):
                    if fol4 == ".DS_Store": continue
                    path4 = os.path.join(path3, fol4)
                    # After going through all of the subfolders, we can finally see the image
                    for img in tqdm(os.listdir(path4), disable=True):
                        if img == ".DS_Store": continue
                        # print(img)
                        try:
                            path5 = os.path.join(path4, img)
                            image_data = nib.load(path5).get_fdata() # get the numbers of the 3D image
                            image_data = np.rot90(image_data) # make it axial
                            """
                            our_slice = image_data[image_data.shape[0]//2] # get the very middle slice
                            our_slice_reshaped = scale_array(our_slice, IMG_SHAPE)
                            images.append(our_slice_reshaped)
                            plt.imshow(our_slice_reshaped)
                            """
                            
                            big_image = np.empty([IMG_SHAPE[0], IMG_SHAPE[1]*NUM_IMAGES, 1])
                            for i in range(NUM_IMAGES):
                                our_slice = image_data[image_data.shape[0]*(i+1)//(NUM_IMAGES+1)]
                                our_slice_reshaped = scale_array(our_slice, IMG_SHAPE)
                                for r in range(IMG_SHAPE[0]): 
                                    for c in range(IMG_SHAPE[1]): 
                                        this_number = our_slice_reshaped[r,c,0]
                                        big_image[r,c+(IMG_SHAPE[1]*i),0] = this_number

                            # print(big_image.shape)
                            # plt.imshow(big_image)
                            images.append(big_image) # add it to the list
                            
                            
                            image_info.append([img, False])
                        except:
                            pass
    print("Got!")
    return images, image_info

def scale_array(x, new_size):
    new_size = (new_size[0]/x.shape[0],new_size[1]/x.shape[1], 1)
    y = scipy.ndimage.zoom(x, new_size)
    return y

In [5]:
def create_mlp(dim, regress=False, verbose=True):
    if (verbose): print("Creating MLP Classifier for attributes...")
    # define our MLP network
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(4, activation="relu"))
    # check to see if the regression node should be added
    if regress:
        model.add(Dense(1, activation="linear"))
    # return our model
    if (verbose): print("Created!")
    return model

def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False, verbose=True):
    if (verbose): print("Creating CNN for images...")
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (width, height, depth)
    chanDim = -1
    # define the model input
    inputs = Input(shape=inputShape)
    # loop over the number of filters
    for (i, f) in enumerate(filters):
        # if this is the first CONV layer then set the input
        # appropriately
        if i == 0:
            x = inputs
        # CONV => RELU => BN => POOL
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)
    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(4)(x)
    x = Activation("relu")(x)
    # check to see if the regression node should be added
    if regress:
        x = Dense(1, activation="linear")(x)
    # construct the CNN
    model = Model(inputs, x)
    # return the CNN
    if (verbose): print("Created!")
    return model

In [6]:
def score(model,verbose=False):
    score = 0
    start = int(round(time()*1000))
    pred = model.predict([testNumX, testImagesX])
    end = int(round(time()*1000))
    for i in range(len(pred)):
        prediction = np.where(pred[i] == np.amax(pred[i]))[0][0]
        real = np.where(testY[i] == 1)[0][0]
        # print(pred[i], testY[i])
        if verbose: print("Predicted Class vs. Actual Class: %s %s" % (prediction, real))
        if (prediction == real):
            score+=1
        else: 
            if verbose: print("   Incorrect!")
    return score / len(pred), end-start

In [ ]:
# OPTIONAL

def view_image():
    import numpy as np # linear algebra
    import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    import skimage, os
    from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
    from skimage.measure import label,regionprops, perimeter
    from skimage.morphology import binary_dilation, binary_opening
    from skimage.filters import roberts, sobel
    from skimage import measure, feature
    from skimage.segmentation import clear_border
    from skimage import data
    from scipy import ndimage as ndi
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    import scipy.misc
    import numpy as np
    from glob import glob
    from skimage.io import imread
    BASE_IMG_PATH = "/Users/sidharthanantha/Documents/SciFair 2022/ADNI/002_S_6030/Axial_T2_STAR/2017-06-15_13_30_22.0/S573170/ADNI_002_S_6030_MR_Axial_T2_STAR__br_raw_20170616113815278_41_S573170_I861966.nii"

    image = glob(os.path.join(BASE_IMG_PATH, '*'))
    image = glob(BASE_IMG_PATH)[0]
    print(image)
    %matplotlib inline
    try:
        import nibabel as nib
    except:
        raise ImportError('bip bop')

    image_data = nib.load(image).get_data()
    image_data = np.rot90(image_data)
    # image_data = np.reshape(image_data, (44, 256, 256, 1))
    # image_data = np.rot90(image_data) # make it axial
    # print(image_data.shape)
    le_img = image_data[image_data.shape[0]//2]
    plt.imshow(le_img)
    # print(le_img[len(le_img)//2])
    # print(le_img.shape)
    le_img = scale_array(le_img, (70,70))
    plt.imshow(le_img)
    # print(image_data[2])
    # It's a little smushed but that's ok
view_image()

In [14]:
IMAGE_DIR = "/Users/sidharthanantha/Documents/SciFair 2022/Alzheimers_Dataset_V3"
CSV_DIR = "/Users/sidharthanantha/Documents/SciFair 2022/classical_v2_attr_data.csv"
IMG_SHAPE = (70, 70)
LR = 1e-3
MODEL_NAME = "classifying_alzheimers_v3.model"
Y_ATTR = "Research Group"
INCLUDE_DATA_NUMBERS = ['Weight','APOE A1','APOE A2','Age','MMSE Total Score','GDSCALE Total Score','Global CDR','FAQ Total Score','NPI-Q Total Score']
INCLUDE_DATA_CATEGORIES = ['Sex', 'Description','Type','Imaging Protocol','Structure']
EPOCHS = 200
NUM_IMAGES = 3
RG_CLASSIFICATIONS = {
    "CN": 0,
    "AD": 1,
    "MCI": 2,
    "EMCI": 2,
    "LMCI": 2,
    "SMC": 2,
}
NUM_CLASSES = len(list(set(RG_CLASSIFICATIONS.values())))

In [15]:
images_set, image_info = get_data()
print("There are %s images" % len(images_set))

Getting Image Data...
Got!
There are 11855 images


In [16]:
df = process_images()
print("DF's Shape: %s" % str(df.shape))

Processing Images...
Processed!
DF's Shape: (2324, 22)


In [130]:
df

,Subject ID,Sex,Weight,Research Group,APOE A1,APOE A2,Visit,Study Date,Age,Global CDR,...,GDSCALE Total Score,FAQ Total Score,Modality,Description,Type,Imaging Protocol,Image ID,Structure,Registration,Downloaded
9,002_S_0295,M,72.6,0,3.0,4.0,ADNI1/GO Month 12,5/25/2007,86.0,0.0,...,0.0,0.0,MRI,3-plane localizer,Original,Slice Thickness=5.0;Matrix Z=5.0;Acquisition T...,55271,Brain,native,2222
11,002_S_0295,M,72.6,0,3.0,4.0,ADNI1/GO Month 12,5/25/2007,86.0,0.0,...,0.0,0.0,MRI,Axial PD/T2 FSE,Original,Slice Thickness=3.0;Matrix Z=51.0;Acquisition ...,55277,Brain,native,2221
13,002_S_0295,M,73.8,0,3.0,4.0,ADNI1/GO Month 24,7/23/2008,87.2,0.0,...,0.0,0.0,MRI,Axial PD/T2 FSE,Original,Slice Thickness=3.0;Matrix Z=48.0;Acquisition ...,114208,Brain,native,2220
34,002_S_0413,F,60.4,0,3.0,3.0,ADNI Baseline,5/19/2006,76.4,0.0,...,0.0,0.0,MRI,Double_TSE,Original,Slice Thickness=3.0;Matrix Z=52.0;Manufacturer...,15804,Brain,native,10459
38,002_S_0413,F,60.8,0,3.0,3.0,ADNI1/GO Month 6,11/15/2006,76.9,0.0,...,0.0,0.0,MRI,Axial PD/T2 FSE,Original,Slice Thickness=3.0;Matrix Z=52.0;Acquisition ...,29707,Brain,native,153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50388,941_S_4377,F,123.4,4,3.0,4.0,ADNI2 Month 6-New Pt,8/16/2012,70.1,0.5,...,2.0,2.0,MRI,localizer,Original,Slice Thickness=8.0;Matrix Z=1.0;Acquisition T...,324374,Brain,native,10328
50389,941_S_4377,F,123.4,4,3.0,4.0,ADNI2 Month 6-New Pt,8/16/2012,70.1,0.5,...,2.0,2.0,MRI,localizer,Original,Slice Thickness=8.0;Matrix Z=1.0;Acquisition T...,324372,Brain,native,10329
50393,941_S_4377,F,123.4,4,3.0,4.0,ADNI2 Month 6-New Pt,8/16/2012,70.1,0.5,...,2.0,2.0,MRI,AXIAL_T2_STAR,Original,Slice Thickness=4.0;Matrix Z=44.0;Acquisition ...,324377,Brain,native,10327
50420,941_S_4420,M,75.3,3,3.0,3.0,ADNI2 Month 6-New Pt,12/21/2012,82.2,0.5,...,4.0,1.0,MRI,ADNI_gre_field_mapping,Original,Slice Thickness=3.0;Matrix Z=45.0;Acquisition ...,352803,Brain,native,363


In [17]:
images = []

for index in list(df["Downloaded"]):
    images.append(images_set[index])
images = np.array(images)
print("Shape of the Image array: %s" % str(np.shape(images)))

Shape of the Image array: (2324, 70, 70, 1)


In [18]:
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

trainY = to_categorical(np.array(trainAttrX[Y_ATTR]))
testY = to_categorical(np.array(testAttrX[Y_ATTR]))
trainAttrX

trainNumX = process_lattributes(df, trainAttrX)
testNumX = process_lattributes(df, testAttrX)

Processing Scan Attributes...
Processed!
Processing Scan Attributes...
Processed!


In [21]:
def make(verbose=True):
    mlp = create_mlp(trainNumX.shape[1], regress=False, verbose=verbose)
    cnn = create_cnn(*np.shape(trainImagesX[0]), regress=False, verbose=verbose)
    combinedInput = concatenate([mlp.output, cnn.output])

    x = Dense(4, activation="relu")(combinedInput)
    x = Dense(NUM_CLASSES, activation="softmax")(x)
    model = Model(inputs=[mlp.input, cnn.input], outputs=x)
    opt = Adam(learning_rate=1e-3, decay=1e-3 / 200)
    model.compile(loss="categorical_crossentropy", optimizer=opt)
    return model

def train(model, verbose=True):
    if (verbose): print("Fitting...")
    start = int(round(time()*1000))
    model.fit(
        x=[trainNumX, trainImagesX], y=trainY,
        epochs=EPOCHS, batch_size=8, verbose=False)
    end = int(round(time()*1000))
    train_time = (end-start)/1000
    if (verbose): print("Fitted!")
    return model, train_time

def test(model):
    lescore, pred_time = score(model, verbose=False)

    print("After training the model with %s inputs and %s tests, here are the results"%(trainY.shape[0], testY.shape[0]))
    print("-----------")
    print("Time to train (%s inputs, epoch=%s): %s sec (%s mins)" % (trainY.shape[0], EPOCHS, train_time, train_time/60))
    print("Time to predict (%s values) in ms: %s" % (testY.shape[0], pred_time))
    print("Score: %s%%" % (lescore*100))

    model.save(MODEL_NAME)
    return lescore*100

In [22]:
NUM_TIMES_TEST = 1

total_scores = 0
for i in range(NUM_TIMES_TEST):
    model = make()
    model, train_time = train(model)
    the_score = test(model)
    total_scores += the_score
    
print("\n\nAvg Scores: %s" % (total_scores/NUM_TIMES_TEST))

Creating MLP Classifier for attributes...
Created!
Creating CNN for images...
Created!
Fitting...
Fitted!
After training the model with 1743 inputs and 581 tests, here are the results
-----------
Time to train (1743 inputs, epoch=200): 4590.473 sec (76.50788333333334 mins)
Time to predict (581 values) in ms: 1387
Score: 97.24612736660929%


Avg Scores: 97.24612736660929
